In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [205]:
%run -i functions.py

Functions Loaded!


In [206]:
av_raw = pd.read_csv("Availability_for_ML.csv")
fa_nov = pd.read_csv("SCS_Alerts_NOV.csv")
fa_oct = pd.read_csv("SCS_Alerts_OCT.csv")

data = pre_process_av_and_fa_oct_nov(av_raw,fa_oct,fa_nov,remove_same_location_faults = True)

fa = data['faults']
av = data['availability']

fa = floor_time(fa,'Entry time',floor_units='H',shift = 10,shift_units='m') 

fa = faults_aggregate_and_pivot(df=fa,
                                time_col='Entry time',
                                fault_level='PLC',
                                agg_col='Duration',
                                agg_type='count',#can use mean/count/sum
                                quadrant=None,# by default all quadrants are kept
                                break_durations=False)

fa  = weight_hours(df=fa, weights = [1])

av = availability_quadrant_mean(df=av,
                                time_col='Datetime',
                                level = 'Module', # by default all data is kept
                                selection = 13) 

df = merge_av_fa(av_df=av,
                 fa_df=fa,
                 min_date=None, 
                 max_date=None) #specify min/max date if you want to filter data by date


#remove columns with only zeros (faults that did not happen in this period of time or quadrant)
df = df.loc[:, (df != 0).any(axis=0)]
df.head()

Quadrants Assigned
Modules Assigned
duplicated location faults removed - max duration kept
Fault and availability data pre-processed
Time shifted by 10m
Faults aggregated and pivoted
Previous Hours Weighted
Output will contain data only for Module: 13
Availability data aggregated
Availability and fault datasets merged


,Datetime,Downtime,C05,C06,C07,C08,C09,C10,C11,C12,...,SCSM11,SCSM12,SCSM13,SCSM14,SCSM15,SCSM17,SCSM18,SCSM19,SCSM20,SCSM22
0,2020-10-16 06:00:00,0.409765,4.0,6.0,5.0,19.0,14.0,1.0,11.0,3.0,...,2.0,3.0,2.0,1.0,7.0,2.0,0.0,0.0,2.0,1.0
1,2020-10-16 07:00:00,0.180526,5.0,4.0,6.0,24.0,17.0,1.0,7.0,4.0,...,16.0,12.0,0.0,0.0,4.0,1.0,1.0,0.0,2.0,4.0
2,2020-10-16 08:00:00,0.164189,8.0,12.0,4.0,31.0,9.0,6.0,9.0,13.0,...,1.0,12.0,1.0,0.0,2.0,0.0,4.0,7.0,1.0,3.0
3,2020-10-16 09:00:00,0.180914,18.0,13.0,7.0,13.0,14.0,1.0,8.0,10.0,...,6.0,3.0,0.0,1.0,3.0,3.0,2.0,1.0,2.0,9.0
4,2020-10-16 10:00:00,0.344868,16.0,12.0,13.0,17.0,16.0,5.0,11.0,19.0,...,4.0,15.0,0.0,0.0,0.0,2.0,6.0,0.0,1.0,2.0
